In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

2024-08-10 22:42:35.070867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 22:42:35.087403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 22:42:35.087451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-10 22:42:35.096570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 22:42:35.605275: W tensorflow/compiler/tf

In [4]:
# GPU 장치 목록 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow는 GPU를 사용할 수 있습니다.")
    for gpu in gpus:
        print(f"사용 가능한 GPU: {gpu}")

    # gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    # print(gpus)
    # if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)
else:
    print("TensorFlow는 GPU를 사용할 수 없습니다.")

TensorFlow는 GPU를 사용할 수 있습니다.
사용 가능한 GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-08-10 22:42:36.144835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-10 22:42:36.166524: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-10 22:42:36.166569: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [6]:
# 데이터 전처리
x_train = x_train.reshape(-1, 32, 32, 3).astype('float32') / 255.0
x_test = x_test.reshape(-1, 32, 32, 3).astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [152]:
inputs = tf.keras.Input(shape=(32, 32, 3))

# 첫 번째 블록
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu')(inputs)
# x = tf.keras.layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

# 두 번째 블록 (MLPConv 레이어)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=16, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

# 세 번째 블록 (MLPConv 레이어)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=192, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

# 내 번째 블록 (MLPConv 레이어)
# x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(x)
# x = tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 1), padding='same', activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(100, activation='softmax')(x)

In [153]:
# 모델 생성
model = tf.keras.Model(inputs=inputs, outputs=x)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_36 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_307 (Conv2D)             │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_70 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_308 (Conv2D)             │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_309 (Conv2D)             │ (None, 13, 13, 64)     │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_71 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_310 (Conv2D)             │ (None, 6, 6, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_311 (Conv2D)             │ (None, 6, 6, 128)      │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 6, 6, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_72 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_36     │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129,252 (504.89 KB)

 Trainable params: 128,804 (503.14 KB)

 Non-trainable params: 448 (1.75 KB)

In [154]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0902 - loss: 4.1656 - val_accuracy: 0.1108 - val_loss: 4.1019
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2357 - loss: 3.1481 - val_accuracy: 0.2581 - val_loss: 3.0507
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3068 - loss: 2.7401 - val_accuracy: 0.2541 - val_loss: 3.1003
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3610 - loss: 2.4841 - val_accuracy: 0.2214 - val_loss: 3.3821
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4033 - loss: 2.2868 - val_accuracy: 0.3144 - val_loss: 2.7600
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4399 - loss: 2.1312 - val_accuracy: 0.3063 - val_loss: 2.9781
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4742 - loss: 1.9982 - val_accuracy: 0.3437 - val_loss: 2.6344
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4935 - loss: 1.8809 - val_accuracy: 0.

In [155]:
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3786 - loss: 2.5546
Test Loss: 2.5462
Test Accuracy: 0.3751


In [156]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from PIL import Image

# 이미지 로드 및 전처리
def prepare_image(img_path, target_size=(32, 32)):
    # 이미지를 열고 리사이즈
    img = Image.open(img_path)
    img = img.resize(target_size)
    
    # 이미지를 numpy 배열로 변환하고 차원 추가
    img_array = np.array(img)
    
    # 색상 채널이 3개인지 확인
    if img_array.shape[-1] != 3:
        img_array = np.stack([img_array] * 3, axis=-1)
    
    img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
    img_array = img_array.astype('float32') / 255.0  # 정규화
    return img_array

# 예측 함수
def predict_image(model, img_path):
    img_array = prepare_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0], predictions

# 예측할 이미지 경로
img_path = 'apple.png'

# 예측 수행
predicted_class, predictions = predict_image(model, img_path)

cifar100_class_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beetle', 'bicycle', 'bottle', 'bowl',
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar',
    'chair', 'clock', 'cloud', 'cocktail', 'computer', 'couch', 'crab', 'cup',
    'dinosaur', 'dog', 'drum', 'elephant', 'flatfish', 'forest', 'fox', 'girl',
    'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'mailbox', 'man', 'map', 'motorcycle', 'mountain', 'mouse', 'mushroom',
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree',
    'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray',
    'road', 'rocket', 'rose', 'seal', 'shark', 'shoes', 'skateboard', 'skis',
    'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper',
    'table', 'tank', 'telephone', 'television', 'tiger', 'toilet', 'train', 'truck',
    'umbrella', 'waterfall', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]
print(cifar100_class_names[predicted_class])
print(f"Predicted Class: {predicted_class}")
print(f"Predictions: {predictions}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
television
Predicted Class: 83
Predictions: [[1.35430023e-01 8.39571101e-07 1.00743819e-06 1.11924701e-08
  4.20343547e-08 6.18094873e-06 2.32851480e-08 2.40499885e-05
  4.87393859e-09 2.40078208e-07 8.95340985e-04 9.47573335e-06
  3.06416559e-10 6.27219805e-08 1.17378067e-05 9.16298404e-07
  5.83336107e-04 4.59413826e-11 4.39280348e-06 1.61452263e-07
  1.01702881e-06 4.41120562e-09 2.17689259e-04 3.73397873e-13
  1.85442957e-06 4.84917218e-05 2.34069172e-04 1.02449285e-07
  3.11174904e-06 2.43909597e-07 9.54591339e-09 1.79392404e-10
  7.54317007e-05 7.50800117e-11 5.19197352e-09 3.81828329e-07
  3.15769788e-09 5.00859043e-09 1.39667664e-08 4.73071050e-05
  3.84378336e-06 1.91914501e-06 2.28772690e-09 1.36669648e-10
  1.80172719e-06 2.15014457e-04 2.37623610e-07 5.80604826e-08
  2.45405658e-07 6.77127077e-10 5.06217248e-07 7.56879004e-08
  9.76147829e-10 1.97453392e-05 2.47645276e-05 2.73304277e-08
  4.49754622e-10 1.22972578e-03 1.13108490e-06 1.